In [1]:
import numpy as np
import tensorflow as tf

In [2]:
experiment = '_2snap'

_04847_img = np.load('4847' + experiment + '-image.npy')
_04799_img = np.load('4799' + experiment + '-image.npy')
_04820_img = np.load('4820' + experiment + '-image.npy')
_05675_img = np.load('5675' + experiment + '-image.npy')
_05680_img = np.load('5680' + experiment + '-image.npy')
_05710_img = np.load('5710' + experiment + '-image.npy')

_04847_lbl = np.load('4847' + experiment + '-label-onehot.npy')
_04799_lbl = np.load('4799' + experiment + '-label-onehot.npy')
_04820_lbl = np.load('4820' + experiment + '-label-onehot.npy')
_05675_lbl = np.load('5675' + experiment + '-label-onehot.npy')
_05680_lbl = np.load('5680' + experiment + '-label-onehot.npy')
_05710_lbl = np.load('5710' + experiment + '-label-onehot.npy')

train_img = np.vstack((_04847_img[5:,], _04799_img[5:,], _04820_img[5:,], _05675_img[5:,], _05680_img[5:,]))
train_lbl = np.vstack((_04847_lbl[5:,], _04799_lbl[5:,], _04820_lbl[5:,], _05675_lbl[5:,], _05680_lbl[5:,]))

#val_img = _05710_img
#val_lbl = _05710_lbl

val_img = np.vstack((_04847_img[:5,], _04799_img[:5,], _04820_img[:5,], _05675_img[:5,], _05680_img[:5,]))
val_lbl = np.vstack((_04847_lbl[:5,], _04799_lbl[:5,], _04820_lbl[:5,], _05675_lbl[:5,], _05680_lbl[:5,]))

print 'train_img', train_img.shape
print 'train_lbl', train_lbl.shape
print 'val_img', val_img.shape
print 'val_lbl', val_lbl.shape

train_img (775, 64, 64, 8)
train_lbl (775, 2)
val_img (25, 64, 64, 8)
val_lbl (25, 2)


In [3]:
def next_batch(X, y, size):
    perm = np.random.permutation(X.shape[0])
    for i in np.arange(0, X.shape[0], size):
        yield (X[perm[i:i + size]], y[perm[i:i + size]])


def weight_variable(shape, name='W'):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name=name)


def bias_variable(shape, name='b'):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name=name)


def loss_accuracy(cross_entropy_count, accuracy_count, x, y_, keep_prob, phase_train, X, Y, batch_size):
    c, l = 0, 0
    for batch_xs, batch_ys in next_batch(X, Y, batch_size):
        feed_dict = {x: batch_xs, y_: batch_ys, keep_prob: 1.0, phase_train: False}
        l += cross_entropy_count.eval(feed_dict=feed_dict)
        c += accuracy_count.eval(feed_dict=feed_dict)
    return float(l) / float(Y.shape[0]), float(c) / float(Y.shape[0])

In [4]:
def conv_layer(input, channels_in, channels_out, name='conv', patch_x=15, patch_y=15):
    with tf.name_scope(name):
        # 15x15 patch, 8 channels, 32 output channels
        w = weight_variable([patch_x, patch_y, channels_in, channels_out], name='W')
        b = bias_variable([channels_out], name='B')
        conv = tf.nn.conv2d(input, w, strides=[1, 1, 1, 1], padding='SAME') + b
        bn = batch_norm(conv, channels_out, phase_train, name + '_bn')
        act = tf.nn.relu(bn)
        #act = tf.nn.relu(conv)
#         tf.summary.histogram('weights', w)
#         tf.summary.histogram('biases', b)
#         tf.summary.histogram('activations', act)
        return tf.nn.max_pool(act, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
def fc_layer(input, channels_in, channels_out, name='fc'):
    with tf.name_scope(name):
        w = weight_variable([channels_in, channels_out], name='W')
        b = bias_variable([channels_out], name='b')
        bn = tf.matmul(input, w) + b
#         tf.summary.histogram('weights', w)
#         tf.summary.histogram('biases', b)
        return tf.nn.relu(bn)

In [5]:
def batch_norm(x, n_out, phase_train, name='bn'):
    """
    Batch normalization on convolutional maps.
    Ref.: http://stackoverflow.com/questions/33949786/how-could-i-use-batch-normalization-in-tensorflow
    Args:
        x:           Tensor, 4D BHWD input maps
        n_out:       integer, depth of input maps
        phase_train: boolean tf.Varialbe, true indicates training phase
        scope:       string, variable scope
    Return:
        normed:      batch-normalized maps
    """
    with tf.variable_scope(name):
        beta = tf.Variable(tf.constant(0.0, shape=[n_out]), name='beta', trainable=True)
        gamma = tf.Variable(tf.constant(1.0, shape=[n_out]), name='gamma', trainable=True)
        batch_mean, batch_var = tf.nn.moments(x, [0,1,2], name='moments')
        ema = tf.train.ExponentialMovingAverage(decay=0.6)

        def mean_var_with_update():
            ema_apply_op = ema.apply([batch_mean, batch_var])
            with tf.control_dependencies([ema_apply_op]):
                return tf.identity(batch_mean), tf.identity(batch_var)

        mean, var = tf.cond(phase_train,
                            mean_var_with_update,
                            lambda: (ema.average(batch_mean), ema.average(batch_var)))
        normed = tf.nn.batch_normalization(x, mean, var, beta, gamma, 1e-2)
    return normed

In [6]:
x = tf.placeholder(tf.float32, shape=[None, 64, 64, 8], name='x')
x_image = tf.reshape(x, [-1, 64, 64, 8], name='x_reshaped')
y_ = tf.placeholder(tf.float32, shape=[None, 2], name='labels')
phase_train = tf.placeholder(tf.bool, name='phase_train')
keep_prob = tf.placeholder(tf.float32, name='keep_prob_fc1')

conv1 = conv_layer(x_image, 8, 32, 'conv1', 15, 15) # 15x15 patch, 8 channels, 32 output channels
conv2 = conv_layer(conv1, 32, 64, 'conv2', 15, 15) # 15x15 patch, 32 channels, 64 output channels
conv2_flat = tf.reshape(conv2, [-1, 16 * 16 * 64]) # flatten conv2

fc1 = fc_layer(conv2_flat, 16 * 16 * 64, 1024, 'fc1') # fc_layer fc1
fc1_drop = tf.nn.dropout(fc1, keep_prob) # dropout on fc1
fc2 = fc_layer(fc1_drop, 1024, 512, name='fc2') # fc_layer fc2

with tf.name_scope('fc3'):
    W_fc3 = weight_variable([512, 2])
    b_fc3 = bias_variable([2])
    
    tf.summary.histogram('weights', W_fc3)
    tf.summary.histogram('biases', b_fc3)
    
    y_conv = tf.matmul(fc2, W_fc3) + b_fc3

In [ ]:
max_epochs = 200
max_steps = 1000
learning_rate = 1e-5
step, train_acc_arr, train_loss_arr, val_acc_arr, val_loss_arr = 0, [], [], [], []
with tf.name_scope('xent'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
    cross_entropy_count = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    accuracy_count = tf.reduce_sum(tf.cast(correct_prediction, tf.float32))

tf.summary.scalar('cross_entropy', cross_entropy)
tf.summary.scalar('accuracy', accuracy)
# tf.summary.image('input', x_image, 3)

<tf.Tensor 'accuracy_1:0' shape=() dtype=string>

In [ ]:
sess = tf.InteractiveSession()
merged_summary = tf.summary.merge_all()
writer = tf.summary.FileWriter('/tmp/brain/4')
writer.add_graph(sess.graph)

sess.run(tf.global_variables_initializer())

for i in range(max_epochs):
    for batch_xs, batch_ys in next_batch(train_img, train_lbl, 5):
        feed_dict={x: batch_xs, y_: batch_ys, keep_prob: 0.5, phase_train: True}
        if step % 5 == 0:
            s = sess.run(merged_summary, feed_dict=feed_dict)
            writer.add_summary(s, step)

        train_step.run(feed_dict=feed_dict)
        step += 1

        if step % 100 == 0:
            train_loss, train_accuracy = loss_accuracy(cross_entropy_count, accuracy_count, x, y_, keep_prob, phase_train, train_img, train_lbl, 100)
            val_loss, val_accuracy = loss_accuracy(cross_entropy_count, accuracy_count, x, y_, keep_prob, phase_train, val_img, val_lbl, 100)
            
            train_acc_arr.append(train_accuracy), train_loss_arr.append(train_loss)
            val_acc_arr.append(val_accuracy), val_loss_arr.append(val_loss)

            print "step %d, train accuracy %.4f, train loss %.5f, val accuracy %g, val loss %g" % (
                step, train_accuracy, train_loss, val_accuracy, val_loss)

step 100, train accuracy 0.4994, train loss 4.40857, val accuracy 0.68, val loss 3.57852
step 200, train accuracy 0.5058, train loss 5.21697, val accuracy 0.6, val loss 3.55853
step 300, train accuracy 0.5265, train loss 3.77261, val accuracy 0.56, val loss 3.66727
step 400, train accuracy 0.5213, train loss 5.28156, val accuracy 0.68, val loss 4.26625
step 500, train accuracy 0.5523, train loss 2.93206, val accuracy 0.56, val loss 4.02645
step 600, train accuracy 0.5548, train loss 3.13437, val accuracy 0.4, val loss 4.50203
step 700, train accuracy 0.5458, train loss 3.42871, val accuracy 0.52, val loss 3.72256
step 800, train accuracy 0.5535, train loss 2.46558, val accuracy 0.56, val loss 3.33611
step 900, train accuracy 0.5587, train loss 2.36939, val accuracy 0.44, val loss 3.2901
step 1000, train accuracy 0.5652, train loss 4.00541, val accuracy 0.4, val loss 5.46835
step 1100, train accuracy 0.5794, train loss 2.58597, val accuracy 0.48, val loss 2.65572
step 1200, train accura

In [ ]:
test_loss, test_accuracy = loss_accuracy(cross_entropy_count, accuracy_count, x, y_, keep_prob, phase_train,
                                         val_img, val_lbl, 100)
print 'Val Accuracy: %g, Val Loss: %g' % (test_accuracy * 100, test_loss)

In [ ]:
from tensorflow.python.client import device_lib
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
get_available_devices()

In [ ]:
val_img.shape

In [ ]:
val_img[1,:,:,1].shape

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
for i in range(8):
    plt.show(plt.imshow(val_img[1,:,:,i]))

In [ ]:
plt.show(plt.imshow(np.concatenate(val_img[1], axis=1)))

In [ ]:
plt.show(plt.imshow(np.concatenate((val_img[1,:,:,0], val_img[1,:,:,1], val_img[1,:,:,2], val_img[1,:,:,3], val_img[1,:,:,4], val_img[1,:,:,5], val_img[1,:,:,6], val_img[1,:,:,7]), axis=1)))

In [ ]:
plt.show(plt.imshow(np.concatenate(val_img[1,:,:,[0,1,2,3,4,5,6,7]], axis=1)))

In [ ]:
val_img[1].T.T.T.shape